# Datasource Data Processing for NRI and Air Quality Database
Harshavardhini R harshar@uw.edu

11/1/2021

## I. tbl_dataset_info

In [1]:
import pandas as pd
import numpy as np
import os, glob
import zipfile

### C. NRI_Table_Census County & Tracts - dataset_id 3

In [20]:
# county
NRI_county = pd.read_csv("Datasource/dataset3_NRI/NRI_Table_Counties/NRI_Table_Counties.csv")
NRI_census_tract = pd.read_csv("Datasource/dataset3_NRI/NRI_Table_CensusTracts/NRI_Table_CensusTracts.csv")
NRI_test_county = NRI_county.loc[NRI_county['STATE']
NRI_test_county = NRI_test_county.loc[NRI_test_county['COUNTY']


In [21]:
# tracts
NRI_test_tracts = NRI_census_tract.loc[NRI_census_tract['STATE']
NRI_test_tracts = NRI_test_tracts.loc[NRI_test_tracts['COUNTY']


In [22]:
NRI_data = NRI_test_county.append(NRI_test_tracts,ignore_index=True)

In [23]:
NRI_data['geo_id'] = 'US'+NRI_data['geo_id'].astype(str)
NRI_data['year']=2020
NRI_data['variable']="National Risk Index Score"
NRI_data['dataset_id']= 3

In [24]:
dataset3_NRI = NRI_data.reindex(columns=['dataset_id','geo_id','year','variable','value'])

### E. Median Air Quality by County - dataset_id 6

In [57]:
zip_files = glob.glob(os.path.join(mydir+"/Datasource/dataset6_AQI by County 1980-2021", "*.zip"))
zip_files.sort()

#read files in zipfile
all_year_test_county = pd.DataFrame()
for zip_file in zip_files:
    # This is just one file. There are multiple zip files in the folder
    zf = zipfile.ZipFile(zip_file)
    single_year = [pd.read_csv(zf.open(f),usecols=[0,1,2,12]) for f in zf.namelist()]
    single_year = pd.concat(single_year,ignore_index=True)
    single_year_test_county = single_year.loc[(single_year['State']
    all_year_test_county=all_year_test_county.append(single_year_test_county,ignore_index = True)

In [58]:
# add geo_id
all_year_test_county.loc[all_year_test_county['County'].str.contains('Yakima'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[0]
all_year_test_county.loc[all_year_test_county['County'].str.contains('Pierce'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[1]
all_year_test_county.loc[all_year_test_county['County'].str.contains('King'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[2]
all_year_test_county.loc[all_year_test_county['County'].str.contains('Baltimore'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[3]
all_year_test_county.loc[all_year_test_county['County'].str.contains('City'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[4]
all_year_test_county.loc[all_year_test_county['County'].str.contains("Prince George's"), 'geo_id'] = county_county_geo_info['geo_id'].iloc[5]
all_year_test_county.loc[all_year_test_county['County'].str.contains('Montgomery'), 'geo_id'] = county_county_geo_info['geo_id'].iloc[6]

In [59]:
all_year_test_county = all_year_test_county.drop(columns=['State','County'],axis=1)
all_year_test_county = all_year_test_county.assign(dataset_id=6)
all_year_test_county['variable'] = 'Median Air Quality'

In [60]:
# pivot columns to row
median_AQ_pivot= all_year_test_county.set_index(['Year','variable','dataset_id','geo_id']).stack().reset_index()
median_AQ_pivot = median_AQ_pivot.rename(columns={0: 'value'}).drop(columns='level_4',axis=1)